In [ ]:
### 해당 코드는 YOLOv5 형식의 클래스를 분류 및 통일하는 코드입니다.
### A1_to_yolov5_conv 혹은 A2_xj2t_conv를 실행한 후에 실행해주세요.
### 처음에 pothole 클래스 정수값을 input()으로 받기 때문에 확인 후 기입해주세요.

### 궁극적으로 pothole을 1개 이상 포함한 파일(txt/이미지)만 남기게 됩니다.
### TXT_class와 images_class 폴더에 들어있는 파일은 변환작업이 끝난 결과물입니다.
### 이 변환은 pothole의 클래스를 0으로 통일하고 pothole이 1개 이상 포함되지 않은
### 파일의 경우는 변환하지 않으며 로그는 CONVERT 폴더를 참고해주세요.

## class0은 pothole만 있는 단일 행 (pothole의 클래스가 0이 아닐 경우는 0으로 변환)
## class1은 pothole 및 타 class도 있는 복수 행 (pothole 외 클래스만 drop하여 활용)
## classNAN은 pothole의 클래스가 존재하지 않는 행 (drop 삭제)



import os
import shutil

# 폴더 경로 설정
txt_dir = "./TXT"
images_dir = "./images"
convert_dir = "./CONVERT"
class0_path = os.path.join(convert_dir, "class0.txt")
class1_path = os.path.join(convert_dir, "class1.txt")
class_nan_path = os.path.join(convert_dir, "classNAN.txt")

# CONVERT 폴더가 없을 경우 생성
os.makedirs(convert_dir, exist_ok=True)

# 사용자 입력을 통해 class0으로 사용할 클래스 ID 설정
class0_id = int(input("class0은 무엇인가요? (예: 2): "))

# 클래스 분류 결과를 저장할 리스트
class0_files = []
class1_files = []
class_nan_files = []

# 클래스 정보 추출 함수
def get_classes_from_file(file_path):
    classes = set()
    with open(file_path, "r") as f:
        for line in f:
            # 각 줄의 첫 번째 값이 클래스 ID이므로 이를 집합에 추가
            class_id = int(line.split()[0])
            classes.add(class_id)
    return classes

# TXT 폴더의 모든 파일을 순회하며 클래스 정보 확인
for filename in os.listdir(txt_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(txt_dir, filename)
        classes = get_classes_from_file(file_path)

        # 클래스 분류
        if classes == {class0_id}:  # class0_id만 있는 경우
            class0_files.append(filename)
        elif class0_id in classes:  # class0_id와 다른 클래스가 섞여 있는 경우
            class1_files.append(filename)
        else:  # class0_id가 없는 경우
            class_nan_files.append(filename)

# 결과 파일 기록
with open(class0_path, "w") as f0, open(class1_path, "w") as f1, open(class_nan_path, "w") as f_nan:
    for file in class0_files:
        f0.write(file + "\n")
    for file in class1_files:
        f1.write(file + "\n")
    for file in class_nan_files:
        f_nan.write(file + "\n")

print(f"클래스 구분 파일들이 {convert_dir}에 생성되었습니다.")
print(f"class0.txt에 {len(class0_files)}개의 파일이 기록되었습니다.")
print(f"class1.txt에 {len(class1_files)}개의 파일이 기록되었습니다.")
print(f"classNAN.txt에 {len(class_nan_files)}개의 파일이 기록되었습니다.")

# 분류된 파일을 저장할 폴더 생성
txt_class_dir = "./TXT_class"
images_class_dir = "./images_class"
os.makedirs(txt_class_dir, exist_ok=True)
os.makedirs(images_class_dir, exist_ok=True)

# class0.txt 파일에 기록된 파일들에 대해 .txt 파일과 이미지 파일 복사 및 클래스 ID 변경
for filename in class0_files:
    # .txt 파일 경로 설정
    txt_src_path = os.path.join(txt_dir, filename)
    txt_dst_path = os.path.join(txt_class_dir, filename)

    # class0_id가 0이 아닌 경우 클래스 ID를 0으로 변경하여 저장
    with open(txt_src_path, "r") as src_file:
        lines = src_file.readlines()
        if class0_id != 0:
            # 모든 라인의 class0_id를 0으로 변경
            modified_lines = [f"0 {line.split(' ', 1)[1]}" for line in lines]
        else:
            # class0_id가 0일 경우 변경 없이 그대로 사용
            modified_lines = lines

    # 변경된 내용으로 TXT_class 폴더에 저장
    with open(txt_dst_path, "w") as dst_file:
        dst_file.writelines(modified_lines)

    # 이미지 파일 복사 (이미지 확장자를 추정하여 복사)
    image_name = os.path.splitext(filename)[0]
    for ext in ['.jpg', '.jpeg', '.png', '.bmp', '.gif']:  # 지원할 이미지 확장자
        image_src_path = os.path.join(images_dir, image_name + ext)
        if os.path.exists(image_src_path):
            image_dst_path = os.path.join(images_class_dir, image_name + ext)
            shutil.copy(image_src_path, image_dst_path)
            break  # 이미지 파일을 찾으면 복사 후 종료

# class1.txt에 기록된 파일들에 대해 다른 클래스 행 제거 후 복사
for filename in class1_files:
    # 원본 .txt 파일 경로
    txt_src_path = os.path.join(txt_dir, filename)
    txt_dst_path = os.path.join(txt_class_dir, filename)

    # 해당 .txt 파일에서 클래스 ID가 class0_id와 일치하는 행만 남기기
    with open(txt_src_path, "r") as src_file:
        lines = src_file.readlines()
        filtered_lines = [line for line in lines if int(line.split()[0]) == class0_id]

    # class0_id가 0이 아닌 경우, class0_id를 0으로 변경
    if class0_id != 0:
        filtered_lines = [f"0 {line.split(' ', 1)[1]}" for line in filtered_lines]

    # 수정된 .txt 파일을 TXT_class 폴더에 저장
    with open(txt_dst_path, "w") as dst_file:
        dst_file.writelines(filtered_lines)

    # 이미지 파일 복사 (이미지 확장자를 추정하여 복사)
    image_name = os.path.splitext(filename)[0]
    for ext in ['.jpg', '.jpeg', '.png', '.bmp', '.gif']:  # 지원할 이미지 확장자
        image_src_path = os.path.join(images_dir, image_name + ext)
        if os.path.exists(image_src_path):
            image_dst_path = os.path.join(images_class_dir, image_name + ext)
            shutil.copy(image_src_path, image_dst_path)
            break  # 이미지 파일을 찾으면 복사 후 종료

# TXT_class와 images_class 폴더 내 파일 개수 확인
txt_class_count = len(os.listdir(txt_class_dir))
images_class_count = len(os.listdir(images_class_dir))

# 개수 확인 및 출력
print(f"TXT_class 폴더 내 파일 개수: {txt_class_count}")
print(f"images_class 폴더 내 파일 개수: {images_class_count}")

if txt_class_count == images_class_count:
    print("TXT_class와 images_class 폴더의 파일 개수가 동일합니다.")
else:
    print("경고: TXT_class와 images_class 폴더의 파일 개수가 다릅니다.")


클래스 구분 파일들이 ./CONVERT에 생성되었습니다.
class0.txt에 440개의 파일이 기록되었습니다.
class1.txt에 4개의 파일이 기록되었습니다.
classNAN.txt에 47개의 파일이 기록되었습니다.
TXT_class 폴더 내 파일 개수: 444
images_class 폴더 내 파일 개수: 444
TXT_class와 images_class 폴더의 파일 개수가 동일합니다.
